In [50]:
import os 
import chainlit as cl
from langchain import PromptTemplate
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

In [51]:
DATA_PATH = "/Volumes/Transcend/Development/Depresio/backend/ml_models/Chatbot/Knowledge/docs"
DB_FAISS_PATH = "/Volumes/Transcend/Development/Depresio/backend/ml_models/Chatbot/Knowledge/vectorstore/df_faiss"

In [52]:
# def create_vector_db():
#     if not os.path.exists(DB_FAISS_PATH):
#         print("Vector DB already exists")
#     else:
#         loader = DirectoryLoader("DATA_PATH", glob="*.txt")
#         document = loader.load()


#         text_splitter = RecursiveCharacterTextSplitter(
#             chunk_size = 500, 
#             chunk_overlap = 50)

#         texts = text_splitter.split_documents(document)

#         embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
#                                            model_kwargs={'device': 'mps'})
        
#         db = FAISS.from_documents(texts,embeddings)

#         db.save_local(DB_FAISS_PATH)

#         print('vector db created. ')

# create_vector_db()

In [53]:
loader = DirectoryLoader(DATA_PATH, glob="*.txt")
document = loader.load()


text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 500, 
            chunk_overlap = 50)

texts = text_splitter.split_documents(document)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                           model_kwargs={'device': 'mps'})
        
db = FAISS.from_documents(texts,embeddings)

db.save_local(DB_FAISS_PATH)

print('vector db created. ')


2023-10-09 22:15:49 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 3/3 [00:00<00:00, 15.96it/s]


vector db created. 


In [54]:
custom_prompt_template = """Use the following piece of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and noting else.
Helpful answers:
"""

In [55]:
def set_custom_prompt():
    prompt = PromptTemplate(custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [57]:
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever = db.as_retriever(search_kwargs={'k':2}),
        return_source_documents = True,
        chain_type_kwargs={'prompt': prompt}
    )

    return qa_chain


In [ ]:

def load_llm():
    